In [79]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

In [3]:
folder_repository = "/Users/harun/Desktop/Uni/4. Semester/Data Mining/DataMining_TeamA"

folder_data = f"{folder_repository}/resources/data_classification"
folder_labels = f"{folder_repository}/resources/data_classification/labels"


In [34]:
data = pd.read_csv(f"{folder_data}/x2.csv")
data_labels = pd.read_json(f"{folder_data}/labels/y2.json")

labels = []
# add labels to dataset
for i in range(0, len(data_labels)):
    labels.append(data_labels.iloc[i].item())

data.iloc[:, -1] = labels
data.to_csv('x2.csv', index=False)

/var/folders/lk/v59w6y8j6853sjnmmcghklvh0000gn/T/ipykernel_18919/1773332542.py:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.iloc[:, -1] = labels


In [85]:
def accuracy(Y_test, predictions, dataset):
    # extract neurons with most activation
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(np.argmax(prediction))

    accuracy = accuracy_score(Y_test, predicted_classes)
    print(f"{dataset}: {accuracy}")


In [86]:
def feature_importance(model):
    # weights of the model
    weights = model.get_weights()

    # weights of first layer do have the most expressiveness, because it is directly connected to the input data
    weights_first_layer = weights[0]

    # number of features
    num_features = weights_first_layer.shape[0]

    # generate feature names
    feature_names = [f"Feature {i+1}" for i in range(num_features)]

    # compute sum of input layer
    feature_importance = np.sum(np.abs(weights_first_layer), axis=1)

    # sort list with decreasing importance
    sorted_features = sorted(zip(feature_importance, feature_names), reverse=True)

    #print(len(weights_first_layer[0]))
    # print
    for importance, feature_name in sorted_features:
        print(f"{feature_name}:, Importance: {importance / 64}")

In [ ]:
def model_arch():
    datasets = ['x0_with_labels.csv', 'x1_with_labels.csv', 'x2_with_labels.csv']
    for dataset in datasets:
        data = pd.read_csv(f"{folder_data}/{dataset}")

        # Split datasets into train and test
        X_train, X_test, Y_train, Y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.25, shuffle = True)

        # labels are intepretated as ordinal, so the model need to know, that the labels are categories
        Y_train_encoded = to_categorical(Y_train)
        Y_test_encoded = to_categorical(Y_test)

        # Create NN model
        model = Sequential()
        model.add(Dense(64, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(30, activation='softmax'))

        # compile model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # train model
        model.fit(X_train, Y_train_encoded, epochs=10, batch_size=32)
        
        predictions = model.predict(X_test)
        accuracy(Y_test, predictions, dataset[:2])
        feature_importance(model)
model_arch()